<a href="https://colab.research.google.com/github/franmazzaro/7506R-2C2023-GRUPO39/blob/main/7506R_TP2_GRUPO39_CHP1_ENTREGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Librerias**

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score

# **Preprocesamiento**

In [5]:
df_criticas = pd.read_csv('/content/sample_data/train.csv')
df_criticas.head(5)

,ID,review_es,sentimiento
0,0,Uno de los otros críticos ha mencionado que de...,positivo
1,1,Una pequeña pequeña producción.La técnica de f...,positivo
2,2,Pensé que esta era una manera maravillosa de p...,positivo
3,3,"Básicamente, hay una familia donde un niño peq...",negativo
4,4,"El ""amor en el tiempo"" de Petter Mattei es una...",positivo


Dividimos nuestro conjunto de train en 70 30

In [7]:
x_train, x_test, y_train, y_test = train_test_split(df_criticas.review_es,
                                                    df_criticas.sentimiento,
                                                    test_size=0.30,
                                                    random_state=25,
                                                    shuffle=True)

Antes de entrear un modelo para clasificacion necesitamos que este entrenamiento sea con valores numericos, y como nosotros tenemos palabras necesitamos convertirlas a vectores

In [8]:
model_NB = make_pipeline(TfidfVectorizer(), MultinomialNB())
model_NB_2 = make_pipeline(CountVectorizer(), MultinomialNB())

model_NB.fit(x_train, y_train)
model_NB_2.fit(x_train, y_train)

categorias_predichas = model_NB.predict(x_test)
categorias_predichas_2 = model_NB_2.predict(x_test)


Utilizamos make_pipeline para aplicarle a nuestros modelos dos metodos distintos para vectorizar y luego el modelo de Naive Bayes Multidimensional. Una vez obtenidos los entrenamos

In [13]:
print(categorias_predichas)
print(categorias_predichas_2)

['positivo' 'positivo' 'positivo' ... 'negativo' 'positivo' 'positivo']
['positivo' 'positivo' 'positivo' ... 'negativo' 'positivo' 'positivo']


In [15]:
print(accuracy_score(y_test, categorias_predichas_2))
print(accuracy_score(y_test, categorias_predichas))
print(precision_score(y_test, categorias_predichas_2, average='micro'))
print(precision_score(y_test, categorias_predichas, average='micro'))

0.8376
0.8534666666666667
0.8376
0.8534666666666667
